In [1]:
import pandas as pd
import numpy as np

In [65]:
# !ls /Users/paxton615/database/alibaba_user/

In [2]:
df = pd.read_csv('/Users/paxton615/database/alibaba_user/users_2m.csv')

In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
df.head()

,user_id,item_id,category_id,status,date,dayofweek,hour
0,309818,4710383,1792277,pv,2017-11-29,2,20
1,309818,1421743,4069500,pv,2017-11-29,2,20
2,309818,800137,1216617,pv,2017-11-29,2,20
3,309818,2493122,1216617,pv,2017-11-29,2,20
4,309818,1461532,3102419,pv,2017-11-30,3,7


In [5]:
df.columns

Index(['user_id', 'item_id', 'category_id', 'status', 'date', 'dayofweek',
       'hour'],
      dtype='object')

In [6]:
df.status.unique()

array(['pv', 'buy', 'fav', 'cart'], dtype=object)

find out users_buy 

data engineering
1. unique user_id
2. count how many item and category of each user
3. count how many status does each user have
4. categorize time and date, logic, RMF
5. find or create a target for ml

In [7]:
df[df['user_id']==309818].item_id.nunique()/len(df[df['user_id']==309818].item_id)

0.9130434782608695

我需要的是每个用户id看过多少item,category,

In [8]:
len(df['user_id'].unique())

29233

In [9]:
user_list = list(df['user_id'].unique())

In [10]:
item_numb = []
for i in user_list:
    item_numb.append({i:df[df['user_id']==i].item_id.nunique()})

keys = []
for i in item_numb:
    keys.append(list(i.keys()))

keys=[i[0] for i in keys]

len(keys)

values = []
for i in item_numb:
    values.append(list(i.values()))

values = [i[0] for i in values]

len(values)

29233

In [11]:
id_item = pd.DataFrame(zip(keys, values), columns=['id','items'])

In [12]:
user_list = list(df['user_id'].unique())

list_dict = []
for i in user_list:
    list_dict.append({i:df[df['user_id']==i].category_id.nunique()})

keys = []
for i in list_dict:
    keys.append(list(i.keys()))

keys=[i[0] for i in keys]

values = []
for i in list_dict:
    values.append(list(i.values()))

values = [i[0] for i in values]

In [13]:
id_cate = pd.DataFrame(zip(keys, values), columns=['id','category'])

In [14]:
data = id_item.merge(id_cate, on='id')

In [15]:
data.head()

,id,items,category
0,309818,63,37
1,309824,57,19
2,309833,135,28
3,309836,83,21
4,309840,88,46


接下来我需要每一个用户的状态数值，多少pv,cart...

In [16]:
df.head()

,user_id,item_id,category_id,status,date,dayofweek,hour
0,309818,4710383,1792277,pv,2017-11-29,2,20
1,309818,1421743,4069500,pv,2017-11-29,2,20
2,309818,800137,1216617,pv,2017-11-29,2,20
3,309818,2493122,1216617,pv,2017-11-29,2,20
4,309818,1461532,3102419,pv,2017-11-30,3,7


In [17]:
status = df[['user_id','status']]

### pivot要好好研究一下，存起来

In [18]:
status['num']=1  #增加一行，方便之后的pivot（sum）用来加总
status

/Users/paxton615/pycharm/projects/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,user_id,status,num
0,309818,pv,1
1,309818,pv,1
2,309818,pv,1
3,309818,pv,1
4,309818,pv,1
...,...,...,...
2748961,445081,pv,1
2748962,445081,pv,1
2748963,445081,pv,1
2748964,445081,pv,1


In [19]:
status_count = pd.pivot_table(status, values='num',index=['user_id'], columns=['status'],aggfunc=np.sum)

In [20]:
status_count.reset_index(inplace=True)
status_count

status,user_id,buy,cart,fav,pv
0,34,NaN,9.0,5.0,32.0
1,42,NaN,7.0,NaN,50.0
2,43,6.0,6.0,13.0,119.0
3,310,1.0,2.0,NaN,69.0
4,311,9.0,NaN,1.0,34.0
...,...,...,...,...,...
29228,445050,NaN,NaN,1.0,15.0
29229,445061,NaN,1.0,NaN,76.0
29230,445062,NaN,NaN,29.0,112.0
29231,445067,4.0,5.0,NaN,45.0


In [21]:
len(status_count.user_id)

29233

In [22]:
status_count.fillna(0,inplace=True)

In [23]:
status_count

status,user_id,buy,cart,fav,pv
0,34,0.0,9.0,5.0,32.0
1,42,0.0,7.0,0.0,50.0
2,43,6.0,6.0,13.0,119.0
3,310,1.0,2.0,0.0,69.0
4,311,9.0,0.0,1.0,34.0
...,...,...,...,...,...
29228,445050,0.0,0.0,1.0,15.0
29229,445061,0.0,1.0,0.0,76.0
29230,445062,0.0,0.0,29.0,112.0
29231,445067,4.0,5.0,0.0,45.0


In [24]:
data = data.merge(status_count, left_on='id', right_on='user_id')

In [25]:
data.drop('user_id', inplace=True, axis=1)

In [26]:
data.head()

,id,items,category,buy,cart,fav,pv
0,309818,63,37,1.0,6.0,1.0,61.0
1,309824,57,19,5.0,7.0,0.0,53.0
2,309833,135,28,3.0,0.0,29.0,129.0
3,309836,83,21,1.0,2.0,0.0,99.0
4,309840,88,46,1.0,24.0,1.0,66.0


In [27]:
len(data.id)

29233

In [28]:
data.sort_values(by='buy', ascending=False)

,id,items,category,buy,cart,fav,pv
5956,337305,136,65,92.0,2.0,0.0,67.0
15543,381745,270,88,45.0,3.0,6.0,316.0
12757,368575,88,43,43.0,6.0,0.0,77.0
8205,347596,90,69,40.0,12.0,0.0,46.0
24253,42193,176,85,39.0,20.0,1.0,176.0
...,...,...,...,...,...,...,...
21188,407585,17,13,0.0,0.0,0.0,18.0
21189,407586,271,20,0.0,11.0,1.0,304.0
9881,355325,37,23,0.0,1.0,1.0,41.0
9875,355291,119,48,0.0,4.0,1.0,129.0


In [29]:
data.corr()

,id,items,category,buy,cart,fav,pv
id,1.000000,-0.000476,0.007359,0.009427,-0.000628,-0.000317,-0.000515
items,-0.000476,1.000000,0.795002,0.226914,0.469748,0.338314,0.982994
category,0.007359,0.795002,1.000000,0.293655,0.421924,0.300736,0.750067
buy,0.009427,0.226914,0.293655,1.000000,0.214197,0.064262,0.216656
cart,-0.000628,0.469748,0.421924,0.214197,1.000000,0.015237,0.423289
fav,-0.000317,0.338314,0.300736,0.064262,0.015237,1.000000,0.273765
pv,-0.000515,0.982994,0.750067,0.216656,0.423289,0.273765,1.000000


take rmf from previous analysis, combine with data here

In [30]:
rmf=pd.read_csv('/Users/paxton615/database/alibaba_user/rmf.csv')
rmf.drop('Unnamed: 0',inplace=True,axis=1)
rmf.head()

merge data with rmf

In [ ]:
data =data.merge(rmf, left_on='id', right_on='user_id', how='left')

drop 'user_id, recent_shopping, R , F'

In [ ]:
data.drop(['recent_shopping','user_id','R','F'],axis=1, inplace=True)

data.head()

fillna with data['R+F'].mean(), round to 2 digits

In [46]:
data['R+F']=data['R+F'].fillna(data['R+F'].mean())

data['R+F']=[round(i,1) for i in data['R+F']]

In [61]:
data.corr()

,id,items,category,buy,cart,fav,pv,R+F
id,1.000000,-0.000476,0.007359,0.009427,-0.000628,-0.000317,-0.000515,-0.006360
items,-0.000476,1.000000,0.795002,0.226914,0.469748,0.338314,0.982994,-0.001800
category,0.007359,0.795002,1.000000,0.293655,0.421924,0.300736,0.750067,0.020687
buy,0.009427,0.226914,0.293655,1.000000,0.214197,0.064262,0.216656,0.278892
cart,-0.000628,0.469748,0.421924,0.214197,1.000000,0.015237,0.423289,0.027675
fav,-0.000317,0.338314,0.300736,0.064262,0.015237,1.000000,0.273765,-0.004356
pv,-0.000515,0.982994,0.750067,0.216656,0.423289,0.273765,1.000000,-0.006951
R+F,-0.006360,-0.001800,0.020687,0.278892,0.027675,-0.004356,-0.006951,1.000000


### basic data engineer complete

In [67]:
data

,id,items,category,buy,cart,fav,pv,R+F
0,309818,63,37,1.0,6.0,1.0,61.0,2.0
1,309824,57,19,5.0,7.0,0.0,53.0,3.0
2,309833,135,28,3.0,0.0,29.0,129.0,3.0
3,309836,83,21,1.0,2.0,0.0,99.0,2.0
4,309840,88,46,1.0,24.0,1.0,66.0,1.0
...,...,...,...,...,...,...,...,...
29228,44506,43,15,4.0,5.0,0.0,40.0,3.0
29229,445061,55,11,0.0,1.0,0.0,76.0,2.4
29230,445062,113,25,0.0,0.0,29.0,112.0,2.4
29231,445067,43,21,4.0,5.0,0.0,45.0,3.0


In [71]:
len(data[data['buy']==0])/len(data.buy)

0.3204597543871652

In [75]:
# data.buy.value_counts()

predict buy or not buy

In [76]:
data['is_buyer']=[0 if i==0 else 1 for i in data['buy']]

In [77]:
data.head()

,id,items,category,buy,cart,fav,pv,R+F,is_buyer
0,309818,63,37,1.0,6.0,1.0,61.0,2.0,1
1,309824,57,19,5.0,7.0,0.0,53.0,3.0,1
2,309833,135,28,3.0,0.0,29.0,129.0,3.0,1
3,309836,83,21,1.0,2.0,0.0,99.0,2.0,1
4,309840,88,46,1.0,24.0,1.0,66.0,1.0,1


In [78]:
data.columns

Index(['id', 'items', 'category', 'buy', 'cart', 'fav', 'pv', 'R+F',
       'is_buyer'],
      dtype='object')

In [ ]:
df = data[['items', 'category', 'cart', 'fav', 'pv', 'R+F']]